<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Bloxorz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import time
import os
import random
import numpy as np

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
# Create a dictionary to save memory later
move = {None: "Initial Block", 0 : "NONE", -1 : "UP", 1 : "DOWN", -2 : "LEFT", 2 : "RIGHT"}
state = {0 : "STANDING", 1 : "LYING_HORIZONTALLY", 2 : "LYING_VERTICALLY", 3 : "SPLIT"}
square = {0 : "NONE", 1 : "WEAK", 2 : "STRONG", 3 : "SOFT_SWICH_TOGGLE", 4 : "SOFT_SWITCH_UNOFFABLE", 5 : "HARD_SWITCH_TOGGLE", 6: "HARD_SWITCH_UNOFFABLE", 7 : "TELEPORT"}
on = {0 : "OFF", 1 : "ON"}

In [ ]:
# Position of the block in the map
class Block:
  def __init__(self, parts, last_move = None):
    self.parts = sorted(parts, key = lambda x : 1000*x[0] + x[1])
    self.state = self.get_state()
    self.last_move = last_move

  def get_state(self):
    row_diff = abs(self.parts[0][0] - self.parts[1][0])
    col_diff = abs(self.parts[0][1] - self.parts[1][1])

    if row_diff == 0 and col_diff == 0: return 0
    elif row_diff == 0 and col_diff == 1: return 1
    elif row_diff == 1 and col_diff == 0: return 2
    else: return 3

  # def is_valid(self):
  #   return self.parts[0][0] >= 0 and self.parts[0][1] >= 0 and self.parts[1][0] >= 0 and self.parts[1][1] >= 0
  
  def move(self, instruction, movable_part = None):
    if instruction == 0: return copy.copy(self)

    child_parts = copy.deepcopy(self.parts)
    # Standing
    if self.state == 0:
      if instruction == -1:
        child_parts[0][0] -= 2
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 2
      elif instruction == -2:
        child_parts[0][1] -= 2
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 2
    # Lying horizontally
    elif self.state == 1:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 2
      else:
        child_parts[0][1] += 2
        child_parts[1][1] += 1
    # Lying vertically
    elif self.state == 2:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 2
      elif instruction == 1:
        child_parts[0][0] += 2
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 1
    #Split
    elif self.state == 3: child_parts[movable_part][abs(instruction) - 1] += np.sign(instruction)

    if self.state == 3: return Block(child_parts, (move[instruction], movable_part))
    else: return Block(child_parts, move[instruction])

  def __eq__(self, other):
    return self.parts == other.parts

In [ ]:
class Special_square:
  def __init__ (self, name, position, affects, on = 0):
    self.name = name
    self.position = position
    self.affects = affects
    self.on = on

  def change(self):
    self.on = 1 - self.on
  
  def print(self):
    # Weak switch
    if self.name == 3: 
      if self.on == 1: print("Weak switch at", self.position, "is now on, squares", self.affects, "are added!")
      else: print("Weak switch at", self.position, "is now off, squares", self.affects, "are removed!")
    if self.name == 4:
      if self.on == 1: print("Strong switch at", self.position, "is now on, squares", self.affects, "are added!")
      else: print("Strong switch at", self.position, "is now off, squares", self.affects, "are removed!")
    if self.name == 5: print("Teleport at", self.position, "splits blocks into", self.affects[0], "and", self.affects[1])

In [ ]:
def draw_map_2D(self,current):
    # Clear screen in Windows systems
    # os.system('cls')
    # Clear screen in Mac/Linux systems
    os.system('clear')
    print("-"*len(map[0])*4)
    for i in range(len(map)):
      print('|',end='')
      for j in range(len(map[0])):
          if current.state == 0 and i == current.i1 and j == current.j1:
              print("", "\033[1;34;47m x", "|", end='')
          elif (current.state == 1 or current.get_state() == 2) and (i == current.i1 and j == current.j1) or (i == current.i2 and j == current.j2):
              print("", "\033[1;34;47m x", "|", end='')
          else:
              print("",map[i][j], "|", end='')
      print()
      print("-"*len(map[0])*4)
    
    time.sleep(1)

In [ ]:
class State:
  def __init__ (self, map, block, specials = None, parent = None):
    self.map = map
    self.block = block
    self.specials = specials
    self.parent = parent
    self.update()

  # Get value of current block's position in the map
  def get_value(self, part):
    return self.map[self.block.parts[part][0]][self.block.parts[part][1]]

  def is_valid(self, block = None):
    if block is None: block = self.block
    if block.parts[0][0] not in range(len(self.map)) or block.parts[1][0] not in range(len(self.map)): 
      return False
    if block.parts[0][1] not in range(len(self.map[0])) or block.parts[1][1] not in range(len(self.map[0])): 
      return False
    if block.state == 0 and self.get_value(0) < 2: 
      return False
    if self.map[block.parts[0][0]][block.parts[0][1]] <= 0 or self.map[block.parts[1][0]][block.parts[1][1]] <= 0: 
      return False

    return True

  def is_goal(self):
    return self.block.state == 0 and self.get_value(0) == 10
  
  def find_special(self, position):
    return next(x for x in self.specials if x.position == position)

  #Check if there is a special squared changes its state and return it
  def special_change(self):
    result = None
    # At least one parts in on weak switch
    if self.get_value(0) == 3 and self.parent.get_value(0) != 3:
      result = self.find_special(self.block.parts[0])
    if self.get_value(1) == 3 and self.parent.get_value(1) != 3:
      result = self.find_special(self.block.parts[1])
    
    # Block is standing on strong switch
    if self.block.state == 0 and self.get_value(0) == 4 and self.parent.get_value(0) != 4:
      result = self.find_special(self.block.parts[0])
    
    # Teleport
    if self.get_value(0) == 5:
      result = self.find_special(self.block.parts[0])
    if self.get_value(1) == 5:
      result = self.find_special(self.block.parts[1])

    return result


  def update(self):
    special_change = self.special_change()
    # print(type(special_change))
    
    if special_change == None or not self.is_valid(): return
    special_change.change()
    
    # Switches
    if special_change.name == 3 or special_change.name == 4: 
      if special_change.on == 1: 
        for pos in special_change.affects: self.map[pos[0]][pos[1]] = 2
      else: 
        for pos in special_change.affects: self.map[pos[0]][pos[1]] = 0
    # Teleport
    else: self.block = Block(special_change.affects)


  def move(self, instruction, movable_part = None): 
    child_block = self.block.move(instruction, movable_part)
    if self.is_valid(child_block): return State(copy.deepcopy(self.map), child_block, copy.deepcopy(self.specials), self)
    else: return None

  def expand(self):
    children = []
    if self.block.state == 3:
      for i in [-2, -1, 2, 1]:
        child0 = self.move(i, 0)
        if child0 is not None: children.append(child0)
        child1 = self.move(i, 1)
        if child1 is not None: children.append(child1)
    else:
      for i in [-2, -1, 2, 1]:
        child = self.move(i)
        if child is not None: children.append(child)
    return children

  def __eq__(self, other):
    return self.map == other.map and self.block == other.block

In [ ]:
level1 = State([[2,2,2,0,0,0,0,0,0,0],
                  [2,2,2,2,2,2,0,0,0,0],
                  [2,2,2,2,2,2,2,2,2,0],
                  [0,2,2,2,2,2,2,2,2,2], 
                  [0,0,0,0,0,2,2,10,2,2], 
                  [0,0,0,0,0,0,2,2,2,0]], Block([[1,1],[1,1]]))

level2 = State([[0,0,0,0,0,0,2,2,2,2,0,0,2,2,2],
                [2,2,2,2,0,0,2,2,4,2,0,0,2,10,2],
                [2,2,3,2,0,0,2,2,2,2,0,0,2,2,2],
                [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                [2,2,2,2,0,0,2,2,2,2,0,0,0,0,0]], 
               Block([[3,1],[3,1]]), 
               [Special_square(3, [2,2], [[4,4], [4,5]]), Special_square(4, [1,8], [[4,10], [4,11]])])

level3 = State([[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
                  [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
                  [2,2,2,2,2,2,2,2,2,0,0,2,2,10,2],
                  [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
                  [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]], Block([[2,1],[2,1]]))

level4 = State([[0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                  [0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                  [2,2,2,2,0,0,0,0,0,2,2,2,0,0],
                  [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                  [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                  [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                  [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                  [0,0,0,0,0,2,10,2,0,0,1,1,2,1],
                  [0,0,0,0,0,2,2,2,0,0,1,1,1,1]], Block([[5,1],[5,1]]))

level5 = State([[0,0,0,0,0,0,0,0,0,0,0,2,2,2,2],
                  [0,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
                  [0,2,2,2,2,0,0,0,0,0,0,0,2,2,2],
                  [0,2,2,2,2,0,0,0,0,0,0,0,0,0,0],
                  [0,2,2,2,2,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,2,2,2,2,2,2,2,2,2,2,0,0],
                  [0,0,0,0,0,0,0,0,0,0,2,2,2,2,2],
                  [2,2,2,0,0,0,0,0,0,0,2,2,2,2,2],
                  [2,10,2,2,2,2,2,2,2,2,2,2,2,0,0],
                  [2,2,2,2,2,0,0,0,0,0,0,0,0,0,0]], Block([[1,12],[1,12]]))

level6 = State([[0,0,0,0,0,2,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,2,0,0,2,2,2,0,0,0,0],
                  [0,0,0,0,0,2,0,0,2,2,2,2,2,0,0],
                  [2,2,2,2,2,2,0,0,0,0,0,2,2,2,2],
                  [0,0,0,0,2,2,2,0,0,0,0,2,2,10,2],
                  [0,0,0,0,2,2,2,0,0,0,0,0,2,2,2],
                  [0,0,0,0,0,0,2,0,0,2,2,0,0,0,0],
                  [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                  [0,0,0,0,0,0,0,2,2,2,0,0,0,0,0]], Block([[3,0],[3,0]]))

level7 = State([[0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [2,2,2,2,5,2,0,0,0,2,2,2,2,10,2],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0]], Block([[4,1],[4,1]]), [Special_square(5, [4,4], [[1,10], [7,10]])])

In [ ]:
class BLOXORZ:
  def __init__ (self, initial_state):
    self.initial_state = initial_state

  def DFShelper(self, stack, visited, loop, max_stack_size):
    while len(stack) > 0:
      max_stack_size = max(len(stack), max_stack_size)
      current = stack.pop()
      if current.block.parts == [[4,4], [4,4]]: print("OK")

      # Just to check if initial state is also goal
      if current.is_goal(): return current, loop, max_stack_size

      visited.append(current)
      children = current.expand()
      for child in children:
        if child.is_goal(): return child, loop, max_stack_size
        if child not in visited: 
          stack.append(child)
             
      loop += 1

    return None, loop, max_stack_size

  def DFS(self):
    goal, loop, max_stack_size = self.DFShelper([self.initial_state], [], 0, 0)
    if goal is None: print("No solution")
    else:
      path = [goal]
      while path[-1].parent is not None: path.append(path[-1].parent)
      path.reverse()
      print("DFS executed after", loop, "loops", "with max stack size", max_stack_size)
      for i in range(len(path)): print("Step", i, ":", path[i].block.last_move, path[i].block.parts)

In [ ]:
test = BLOXORZ(level7)

# weak = state1.find_special([2,2])

# print(weak.affects)
state1 = State([[0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [2,2,2,2,5,2,0,0,0,2,2,2,2,10,2],
                [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0]], Block([[3,1],[3,1]]), [Special_square(5, [4,4], [[1,10], [7,10]])])
level1 = BLOXORZ(state1)

level1.DFS()

DFS executed after 27 loops with max stack size 55
Step 0 : None [[3, 1], [3, 1]]
Step 1 : DOWN [[4, 1], [5, 1]]
Step 2 : RIGHT [[4, 2], [5, 2]]
Step 3 : RIGHT [[4, 3], [5, 3]]
Step 4 : None [[1, 10], [7, 10]]
Step 5 : ('DOWN', 1) [[1, 10], [8, 10]]
Step 6 : ('DOWN', 0) [[2, 10], [8, 10]]
Step 7 : ('DOWN', 0) [[3, 10], [8, 10]]
Step 8 : ('DOWN', 0) [[4, 10], [8, 10]]
Step 9 : ('DOWN', 0) [[5, 10], [8, 10]]
Step 10 : ('DOWN', 0) [[6, 10], [8, 10]]
Step 11 : ('DOWN', 0) [[7, 10], [8, 10]]
Step 12 : RIGHT [[7, 11], [8, 11]]
Step 13 : UP [[6, 11], [6, 11]]
Step 14 : UP [[4, 11], [5, 11]]
Step 15 : RIGHT [[4, 12], [5, 12]]
Step 16 : RIGHT [[4, 13], [5, 13]]
Step 17 : RIGHT [[4, 14], [5, 14]]
Step 18 : UP [[3, 14], [3, 14]]
Step 19 : LEFT [[3, 12], [3, 13]]
Step 20 : DOWN [[4, 12], [4, 13]]
Step 21 : DOWN [[5, 12], [5, 13]]
Step 22 : RIGHT [[5, 14], [5, 14]]
Step 23 : UP [[3, 14], [4, 14]]
Step 24 : LEFT [[3, 13], [4, 13]]
Step 25 : DOWN [[5, 13], [5, 13]]
Step 26 : LEFT [[5, 11], [5, 12]]
S

In [ ]:
class BLOXORZ:
  def __init__(self, map, initial_block, current = None):
    self.map = map
    self.initial_block = initial_block
    self.goal = [(index, row.index(10)) for index, row in enumerate(map) if 10 in row][0]
    self.current = initial_block

  # Get value at part_number of current block
  def get_value(self, part_number):
    return self.map[self.current.parts[part_number][0]][self.current.parts[part_number][1]]

  def is_goal(self, current = None):
    if current == None: current = self.state
    return current.state == 0 and self.map[current.parts[0][0]][current.parts[0][1]] == 10

  # Check if current block is valid in this map
  def is_valid(self, current = None):
    if current == None: current = self.current

    if current.parts[0][0] < 0 or current.parts[1][0] < 0 or current.parts[0][1] < 0 or current.parts[1][1] < 0: 
      return False
    if current.parts[0][0] >= len(self.map) or current.parts[1][0] >= len(self.map) or current.parts[0][1] >= len(self.map[0]) or current.parts[1][1] >= len(self.map[0]): 
      return False

    if current.state == 0: 
      return self.map[current.parts[0][0]][current.parts[0][1]] > 1
    else: 
      return self.map[current.parts[0][0]][current.parts[0][1]] > 0 and self.map[current.parts[1][0]][current.parts[1][1]] > 0

  def expand(self, current = None):
    if current == None: current = self.current
    children = []
    for instruction in [-1,1,-2,2]:
      child = current.move(instruction)
      if self.is_valid(child): children.append(child)
    return children
  
  # Estimate cost to goal, calculated my average of Manhattan distances of 2 part of the block or Chebychev distance
  def cost_to_goal(self, current):
    #Average Manhattan
    average = 1/2 * (abs(current.parts[0][0] - self.goal[0]) + abs(current.parts[1][0] - self.goal[0]) + abs(current.parts[1][0] - self.goal[1]) + abs(current.parts[1][1] - self.goal[1]))
    return 1/average
    # Chebychev
    # max1 = max(abs(current.parts[0][0] - self.goal[0]), abs(current.parts[0][1] - self.goal[1]))
    # max2 = max(abs(current.parts[1][0] - self.goal[0]), abs(current.parts[1][1] - self.goal[1]))
    # return 1/max(max1, max2)

  def move_process(self, process):
    current = copy.copy(self.initial_block)
    for move in process: 
      current = current.move(move)
      if not self.is_valid(current): 
        return -1
    return current


In [ ]:
class Genetic():
  population = []
  prob = []
  def __init__(self, game, chromosome_length, population_capacity):
    self.game = game
    self.chromosome_length = chromosome_length
    self.population_capacity = population_capacity
    self.initPopulation()
    self.probCal()

  def initIndividual(self):
    chromosome = []
    for i in range(self.chromosome_length):
      # if chromosome is empty or having NONE as the last gen, add an arbitrary
      if len(chromosome) == 0 or chromosome[-1] == 0: chromosome.append(random.choice((0,1,2,3,4)))
      # if the last gen is UP, avoid DOWN
      elif chromosome[-1] == 1: chromosome.append(random.choice((0,1,3,4)))
      # if the last gen is DOWN, avoid UP
      elif chromosome[-1] == 2: chromosome.append(random.choice((0,2,3,4)))
      # if the last gen is LEFT, avoid RIGHT
      elif chromosome[-1] == 3: chromosome.append(random.choice((0,1,2,3)))
      # if the last gen is LEFT, avoid RIGHT
      else: chromosome.append(random.choice((0,1,2,4)))
    return chromosome
  
  def initPopulation(self):
    while len(self.population) < self.population_capacity:
      individual = self.initIndividual()
      final = self.game.move_process(individual)
      if final != -1: self.population.append(individual)

  def probCal(self):
    for individual in self.population:
      final = self.game.move_process(individual)
      self.prob.append(self.game.cost_to_goal(final))
    _sum = sum(self.prob)
    for i in range(len(self.prob)): self.prob[i] = self.prob[i] / _sum;

  #Seclect a parent in population
  def select(self):
    rannum = random.uniform(0, 1)
    for i in range (self.population_capacity):
      rannum = rannum - self.prob[i]
      if rannum < 0: return self.population[i-1]

  def cross_over(self):
    parent1 = self.select()
    parent2 = self.select()

    ran_index = random.ranint(0, self.chromosome_length)

    for i in range(ran_index, self.chromosome_length): parent1[i] = parent2[i]

    return parent1

  def mutate(self, chromosome):
     ran_index = random.ranint(0, self.chromosome_length)
     chromosome[ran_index] = random.choice((0,1,2,3,4))

  def populationUpdate(self):
    next_population = []

    # Cross over
    while (len(next_population) < self.population_capacity):
      child = self.cross_over()
      if self.game.move_process(child) != -1: next_population.append(child)
    # It is better no mutation
    
    for child in next_population: self.mutate(child)

    self.population = next_population
    self.probCal

  # Loop for 100 times
  def go(self):
    for i in range(100):
      for individual in self.population:
        if self.game.move_process(individual).is_goal(): return individual
      self.populationUpdate()

    return "Cannot find Solution"